L'objectif du projet est de classifier l'émotion d'une personne à partir de l'expression de son visage.

Le dataset que nous avons choisi est composé de 35 887 images classifiée en 7 catégories [neutre, colère, heureux, dégouté, effrayé, triste, surpris]. Ensuite le dataset est divisé en 2 dossier train et validation.

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

# Import du dataset, la seed correspond à la randomisation des données pour l'entraînement
train_ds = tf.keras.utils.image_dataset_from_directory(
    'C:/Users/helou/Documents/Master_data_ia/M2/S2/DL/serl_project/images/train',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'C:/Users/helou/Documents/Master_data_ia/M2/S2/DL/serl_project/images/validation',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Pour commencer nous avons choisi d'effectuer notre entraînement avec un réseau créé "from scratch", il est composé de 
10 couches.
Le rescaling permet normaliser les pixels pour qu'ils passent de 0 à 255 en 0 à 1.
Les couches de convolution permettent d'extraires les informations des images.
Les couches de pooling vont réduire les dimensions des images traitées.
La couche de flatten permet de passer des couches convolutives aux couches denses.
Les couches denses Permettent d'interpréter les caratéristique des images pour faciliter la classification.

In [ ]:
# Create model with neurons
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])


Afin de compiler ce modèle, nous avons utilisé l'optimiser adam, qui permet de mettre les poids à jour pendant l'entraînement.
La fonction de loss va permettre de calculer l'efficacité de la classification.
Enfin la métrique d'accuracy va nous permettre de faire évoluer notre modèle en fonction des résultats de celle-ci.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Nous avons choisi de réaliser nos entraînements sur 30 époques à des fins de gains de temps pour réaliser différents tests. De plus nous avons constaté que 30 époques étaient largement suffisante pour atteindre un maximum local (souvent à 5-6 époques le modèle n'évolue plus).

In [ ]:
epochs = 30
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

model.save('./deep_learning_model_v2.h5')

Sur ce premier entraînement, on voit clairement un problème d'overfitting, en effet, la précision est extrêment élevée sur le dataset d'entraînement (0.93), tandis que sur celui de validation la valeus va stagner aux alentour de 0.42. De plus la validation loss augmente de plus en plus.

Nous avons donc testé un deuxième modèle, "plus léger" car il comporte une couches de moins. Ici le but est de réduire l'overfit.
On y a ensuite rajouté un dropout pour faire perdre de l'apprentissage aux poids : cela réduit également l'overfit.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),  # Dropout layer
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])

epochs = 30

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

Les résultats ne sont pas bons, on remarque qu'à la troisième époque, on pert déjà de la précision lors de la validation tandis qu'elle augmente pour la précision du train. On retrouve notre soucis d'overfiting.

Dans un soucis de trouver la solution optimale et essayer de résoudre le problème d'overfit, nous avons également tester de construire un modèle à partir d'un modèle pré-entraîné. Ici nous utiliserons ResNet50.

Ici, on laisse le modèle tel-quel, sans modification des poids pendant l'entraînement.
On rajoute à Resnet 4 couches :
    -  Un pooling pour réduire les dimensions de l'image traitée
    -  Une couche dense de 128 neuronnes, appliquant une non-linéarité
    -  Une couche de dropout pour réduire l'overfit en oubliant l'apprentissage de certaines couches
    -  Une couche dense de 7 neuronnes permettant la classification, appliquant une normalisation des données.
Enfin, une fonction de callback est utilisée pour arrêter l'entraînement si la validation loss n'évolue plus ou diminue. Cela permet de gagner du temps et éviter l'overfit

In [ ]:
# Load ResNet50 Model
base_model = tf.keras.applications.ResNet50(
    include_top=False, 
    weights='imagenet', 
    input_shape=(img_height, img_width, 3)
)

# Freeze the base model
base_model.trainable = False

# Create new model on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7, activation='softmax')
])

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

epochs = 30
history = model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping]
)


Ce modèle est celui qui nous a apporté les meilleurs résultats.

On remarque que l'entraînement se termine au bout de 15 époques, la fonction d'early stopping prends effet car le modèle n'évolue plus. On atteint une précision de validation aux alentours de 0.55, ce qui est correcte.
Cependant on pourrait espérer une précision plus élevée en améliorant les paramètres du modèle, notamment sa complexité avec plus de couches ou de neuronnes.

In [ ]:
# Create new model on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
])

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

epochs = 30
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping]
)


Le dernier entraînement, sur une couche beaucoup moins dense, ne permet pas d'atteindre des meilleurs résultats. Au bout de la 3ème époque, la précision stagne et la précision de validation descend en-dessous de 0.4.
On a testé un dropout de 0.3 et également une une densification de 64 (tf.keras.layers.Dense(64, activation='relu')). Mais les résultats ne sont pas meilleurs.

Enfin voici un test sur un autre dataset, le RAF-DB dataset, corportant 15000 images.

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

# Import du dataset, la seed correspond à la randomisation des données pour l'entraînement
train_ds = tf.keras.utils.image_dataset_from_directory(
    'C:/Users/helou/Documents/Master_data_ia/M2/S2/DL/sentiment/DATASET/train',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'C:/Users/helou/Documents/Master_data_ia/M2/S2/DL/sentiment/DATASET/test',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


# Load ResNet50 Model
base_model = tf.keras.applications.ResNet50(
    include_top=False, 
    weights='imagenet', 
    input_shape=(img_height, img_width, 3)
)

# Freeze the base model
base_model.trainable = False

# Create new model on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7, activation='softmax')
])

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

epochs = 30
history = model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping]
)


On remarque que ce dataset est nettement plus performant, atteignant une précision de validation de 0.64

Tableau des résultat par rapport au modèle ayant apporté les meilleurs résultats sur la validation accuracy :

Batch size (1, 32, 64)	NA, | 0,52 | 0,55
Layer number (2, 3)	0,55 | 0,37
Densification des couches (32, 64, 128)	0,32 | 0,47 | 0,52
Dropout (0,1, 0,2, 0.3, 0,5)	0,52 | 0,55 | 0.54 | 0,42


Pour conclure sur les modèles from scratch et ResNet50, on atteint un taux de précision supérieur à 50%, ce qui reste peu car cela représente une chance sur deux de faire une bonne prédiction.
Cela est principalement dû à un overfitting qui nous empêchent d'atteindre des résultats pertinents. D'ailleurs on remarque également qu'un autre dataset est plus performant. Prouvant que celui que nous avons sélectionné ne suffit pas.

En comparaison, sur ce lien on retrouve un exemple d'entraînement https://www.kaggle.com/code/yasserhessein/emotion-recognition-with-resnet50. Ici le modèle comporte plus de couches de traitements qui sont moins denses car elles comportent 32 neuronnes.
Ce modèle atteint une précision de validation de 0.86 ce qui est beaucoup plus que nos modèles.

Cela signifie sûrement qu'un modèle plus complexe avec des couches moins denses permettrait d'obtenir de bons résultats, ainsi qu'un bon dataset.

Maintenant que nous avons présenté nos différents modèles,
Voici un exemple de démo en direct qui pourrait être effectué. L'idée est d'activer la caméra de notre ordinateur pour detecter l'émotion du visage qui sera vu par celle-ci.

In [ ]:
# Load the pre-trained model
model = tf.keras.models.load_model('./deep_learning_model.h5')

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to 180x180 to match the model's expected input
    resized_frame = cv2.resize(frame, (180, 180))

    # Preprocess the image as required by ResNet50
    resized_frame = np.expand_dims(resized_frame, axis=0)  # Add batch dimension
    resized_frame = preprocess_input(resized_frame)  # Preprocess the input as ResNet expects

    # Predict the emotion
    predictions = model.predict(resized_frame)
    emotion_index = np.argmax(predictions)  # Assuming the model outputs a softmax layer

    # Map the predicted labels to emotions
    emotions = ['Happy', 'Sad', 'Angry', 'Surprised', 'Neutral', 'Disgust', 'Fear']  # Update this list as per your model
    emotion = emotions[emotion_index]

    # Display the resulting frame with the predicted emotion
    cv2.putText(frame, f'Emotion: {emotion}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Video', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()
